
Aqui vou iniciar fazendo uma análise exploratória dos dados, a fim de fazer limpezas e tratamentos caso necessário!

# Importando bibliotecas

In [1]:
import pandas as pd

## Explorando dados

In [2]:
# inspecionando os dados dos clientes
df_clientes = pd.read_csv('../dados/dados-brutos/olist_customers_dataset.csv')
df_clientes.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [3]:
df_clientes.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [4]:
df_clientes.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [5]:
def inspeciona_df(df):
    """Função para checar e inspecionar os dados das colunas """
    print(df.isnull().sum())
    print(df.dtypes)

In [6]:
df_pedidos = pd.read_csv('../dados/dados-brutos/olist_orders_dataset.csv')
df_pedidos.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
inspeciona_df(df_pedidos)

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object


In [8]:
print(df_pedidos['order_status'].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


### 🧾 Observações sobre Campos Nulos nas Colunas de Datas

A presença de valores nulos em colunas como `order_delivered_carrier_date`, `order_delivered_customer_date` e `order_approved_at` está diretamente relacionada ao **status do pedido**. Abaixo estão os cenários possíveis e suas respectivas interpretações:

---

#### 📦 Status e Significados

| Status        | Descrição                                                                                      | Impacto nos Campos de Data |
|---------------|-----------------------------------------------------------------------------------------------|-----------------------------|
| **shipped**   | Pedido enviado à transportadora, mas **sem confirmação de entrega ao cliente**.               | Pode ter `NaN` em `order_delivered_customer_date` |
| **processing**| Pedido em processamento, **ainda não faturado**.                                              | Pode ter `NaN` em `order_approved_at` e demais datas |
| **canceled**  | Pedido **cancelado** antes de qualquer progresso.                                             | Possui `NaN` em todas as datas subsequentes |
| **invoiced**  | Pedido **faturado**, porém **ainda não despachado**.                                          | Pode ter `NaN` em datas de envio e entrega |
| **unavailable**| Pedido com **problemas no sistema ou pagamento**.                                             | Pode ter `NaN` em todas as datas |
| **created**   | Pedido **criado**, mas **ainda não aprovado/faturado**.                                       | Pode ter `NaN` em todas as datas |
| **approved**  | Pedido **aprovado**, mas **ainda não enviado à transportadora**.                              | Pode ter `NaN` em `order_delivered_carrier_date` e `order_delivered_customer_date` |

---

> ⚠️ **Importante:** A análise de valores nulos deve sempre considerar o contexto do `order_status`, para evitar interpretações incorretas sobre a logística ou o comportamento do cliente.



In [9]:
df_pedidos_nulos = df_pedidos[df_pedidos.isnull().any(axis=1)]
df_pedidos_nulos.head(5)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00


In [10]:
# importando items do pedido
df_items_pedido = pd.read_csv('../dados/dados-brutos/olist_order_items_dataset.csv')
df_items_pedido.head()


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [11]:
inspeciona_df(df_items_pedido)

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object


In [12]:
# importando meios de pagamentos dos pedidos
df_pagamento_pedido = pd.read_csv('../dados/dados-brutos/olist_order_payments_dataset.csv')
df_pagamento_pedido.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [13]:
inspeciona_df(df_pagamento_pedido)

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object


In [14]:
# importando reviews
df_reviews_pedidos = pd.read_csv('../dados/dados-brutos/olist_order_reviews_dataset.csv')
df_reviews_pedidos.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [15]:
inspeciona_df(df_reviews_pedidos)

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64
review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object


As colunas que armazenam o titulo de comentário e o próprio comentário dos clientes estão vazias por que os próprios clientes não preencheram o form de feedback.

In [16]:
# Calculando a proporçào de reviews
sem_texto = df_reviews_pedidos[df_reviews_pedidos['review_comment_message'].isnull() & df_reviews_pedidos['review_comment_title'].isnull()]
porcentagem_sem_texto = len(sem_texto) / len(df_reviews_pedidos) * 100
print(f'{porcentagem_sem_texto:.2f}% dos clientes não escreveram reviews.')

56.96% dos clientes não escreveram reviews.


In [17]:
# Inspecionando a tabela de produtos
df_produtos = pd.read_csv('../dados/dados-brutos/olist_products_dataset.csv')
df_produtos.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [18]:
inspeciona_df(df_produtos)

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64
product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object


In [19]:
df_produtos_nulos = df_produtos[df_produtos.isnull().any(axis=1)]
df_produtos_nulos.head(12)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,NaN,300.0,35.0,7.0,12.0
244,e10758160da97891c2fdcbc35f0f031d,NaN,NaN,NaN,NaN,2200.0,16.0,2.0,11.0
294,39e3b9b12cd0bf8ee681bbc1c130feb5,NaN,NaN,NaN,NaN,300.0,16.0,7.0,11.0
299,794de06c32a626a5692ff50e4985d36f,NaN,NaN,NaN,NaN,300.0,18.0,8.0,14.0
347,7af3e2da474486a3519b0cba9dea8ad9,NaN,NaN,NaN,NaN,200.0,22.0,14.0,14.0
428,629beb8e7317703dcc5f35b5463fd20e,NaN,NaN,NaN,NaN,1400.0,25.0,25.0,25.0


### 📌 Análise de Valores Nulos nas Colunas de Produtos

Durante a análise da base de dados de produtos, foram identificados valores ausentes (`NaN`) em colunas essenciais para a descrição dos itens, como:

- `product_category_name`
- `product_name_lenght`
- `product_description_lenght`
- `product_photos_qty`

---

#### 🧾 Possíveis causas para os dados ausentes:

1. **Cadastro incompleto**
   - O produto pode ter sido registrado inicialmente apenas com dados básicos (peso e dimensões), sem preenchimento das informações textuais e visuais.

2. **Falha na integração de dados**
   - Problemas técnicos ao importar os dados de sistemas externos ou planilhas podem ter causado a perda dessas informações.

3. **Produto descontinuado ou inativo**
   - Produtos removidos da vitrine ou descontinuados podem ter suas descrições apagadas ou ocultadas, permanecendo na base apenas para referência histórica.

4. **Produto sem movimentação**
   - Itens que nunca foram vendidos ou visualizados podem nunca ter sido revisados ou complementados no cadastro.

5. **Dados anonimizados**
   - Em bases públicas (como datasets educacionais), certas informações são propositalmente removidas para preservar a privacidade ou reduzir o volume de dados.

---

#### ✅ Considerações para tratamento:

> Posteriormente irei identificar a causa certa para esses dados ausentes, é apenas uma explicação básica do que pode estar ocorrendo.

In [20]:
df_produtos_translate = pd.read_csv('../dados/dados-brutos/product_category_name_translation.csv')
df_produtos_translate.head(5)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [21]:
inspeciona_df(df_produtos_translate)

product_category_name            0
product_category_name_english    0
dtype: int64
product_category_name            object
product_category_name_english    object
dtype: object


In [22]:
df_vendedores = pd.read_csv('../dados/dados-brutos/olist_sellers_dataset.csv')
df_vendedores.head(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [23]:
inspeciona_df(df_vendedores)

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object


In [24]:
df_localizacao = pd.read_csv('../dados/dados-brutos/olist_geolocation_dataset.csv')
df_localizacao.head(5)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [25]:
inspeciona_df(df_localizacao)

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64
geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object
